In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

# --- 1. PARAMETERS ---
TICKERS = [
    "PPH",   # Pharma
    "XPH",   # S&P Pharma
    "IHE",   # iShares Pharma
    "IBB",   # Biotech
    "XBI",   # Biotech equal-weight
    "XLV",   # Healthcare sector
    "VHT",   # Vanguard Healthcare
    "SPY",   # Market benchmark
    "VIXY",  # VIX ETF proxy (VIX index isn't downloadable directly via yfinance)
]

START_DATE = "2011-01-04"
END_DATE   = "2025-12-01"


# -----------------------------
# 2. DOWNLOAD MULTI-TICKER DATA
# -----------------------------
df = yf.download(
    tickers=TICKERS,
    start=START_DATE,
    end=END_DATE,
    auto_adjust=False,
    group_by='ticker'
)

# -----------------------------
# 3. FLATTEN MULTIINDEX COLUMNS
# -----------------------------
# Result example: Open_PPH, Close_XBI, Volume_SPY, etc.
df.columns = ['_'.join(col).strip() for col in df.columns.to_flat_index()]

# -----------------------------
# 4. DROP ALL ADJ CLOSE COLUMNS
# -----------------------------
df = df[[c for c in df.columns if "Adj Close" not in c]]


[*********************100%***********************]  9 of 9 completed


In [2]:
# ================================================================
# STEP A — SANITIZE DATAFRAME, INFER TICKERS, BUILD BASE RETURNS
# ================================================================

# 1. Infer tickers from the flattened column names
def infer_tickers_from_df(df):
    """
    Extracts ticker symbols from columns like 'PPH_Close', 'XBI_Open'.
    """
    tickers = sorted({col.split("_")[0] for col in df.columns})
    return tickers

TICKERS = infer_tickers_from_df(df)
print("Detected tickers:", TICKERS)


# 2. Create a clean helper that returns OHLCV column names for a ticker
def ohlcv_cols(df, ticker):
    cols = {
        "open":   f"{ticker}_Open",
        "high":   f"{ticker}_High",
        "low":    f"{ticker}_Low",
        "close":  f"{ticker}_Close",
        "volume": f"{ticker}_Volume",
    }
    # Sanity check (if any missing, skip ticker later)
    if not all(col in df.columns for col in cols.values()):
        return None
    return cols


# 3. Compute **1-day returns for all tickers**
#    (Most features depend on these. They are OHLCV-dependent, so shift(1) later.)
for t in TICKERS:
    cols = ohlcv_cols(df, t)
    if cols is None:
        continue

    df[f"{t}_Return_1d"] = df[cols["close"]].pct_change(1)


# 4. Initialize the global feature registry
#    THIS STAYS EMPTY FOR NOW—later steps will fill it automatically.
feature_registry = {}

print("Base returns created. Registry initialized.")


Detected tickers: ['IBB', 'IHE', 'PPH', 'SPY', 'VHT', 'VIXY', 'XBI', 'XLV', 'XPH']
Base returns created. Registry initialized.


In [6]:
# ======================================================================
# STEP B — MINIMAL FEATURE REGISTRY (EXPLICIT SHIFT RULES)
# ======================================================================

# Registry mapping:
#     feature_name → rule
#
# Where rule ∈ {
#     "shift_1",         # Feature depends on OHLCV(t)
#     "no_shift",        # Feature known before t or already lagged
#     "shift_plus_k"     # Future scheduled events → expand to shift(+1…+horizon)
# }
#
# This registry is filled *incrementally* during feature creation.
# After all features are created, the shift engine interprets these rules
# and applies the correct temporal alignment safely (no leakage).


feature_registry = {}  # global within the pipeline


# --------------------------
# VALID SHIFT RULES
# --------------------------
VALID_RULES = {"shift_1", "no_shift", "shift_plus_k"}


def register_feature(registry: dict, feature_name: str, rule: str):
    """
    Register a feature with an explicit temporal shift rule.

    Parameters
    ----------
    registry : dict
        The global registry storing feature_name → rule mappings.

    feature_name : str
        The new feature’s column name.

    rule : str
        One of:
            - "shift_1"       → use t−1 values instead of t
            - "no_shift"      → feature known before t, no alignment needed
            - "shift_plus_k"  → shift forward by k ∈ [1…horizon] later

    Notes
    -----
    - This function stores *metadata only*, not data.
    - Actual shifting occurs in the shift engine (Step F).
    - No rule inference is used — this prevents mistakes and leakage.
    """

    if rule not in VALID_RULES:
        raise ValueError(
            f"Invalid rule '{rule}' for feature '{feature_name}'. "
            f"Allowed: {VALID_RULES}"
        )

    registry[feature_name] = rule

In [7]:
# ======================================================================
# STEP C — PER-TICKER TECHNICAL FEATURE GENERATORS
# ======================================================================
# All these features depend on today's OHLCV(t), therefore:
#     → They ALWAYS get rule = "shift_1"
# Registry tagging is included in every function.


import numpy as np
import pandas as pd


# ---------------------------------------------------------
# Helper: Determine OHLCV column names for a given ticker
# ---------------------------------------------------------
def ohlcv_cols(df: pd.DataFrame, ticker: str):
    """
    Returns (Open, High, Low, Close, Volume) column names for a ticker.
    Returns None if any are missing.
    """
    cols = [f"{ticker}_{c}" for c in ["Open", "High", "Low", "Close", "Volume"]]
    return cols if all(c in df.columns for c in cols) else None


# ---------------------------------------------------------
# MOMENTUM FEATURES
# ---------------------------------------------------------
def add_momentum_features(df: pd.DataFrame, ticker: str, close: str, registry: dict) -> dict:
    feats = {}

    # Simple returns
    for w in [1, 5, 10, 20]:
        name = f"{ticker}_Return_{w}d"
        feats[name] = df[close].pct_change(w)
        register_feature(registry, name, "shift_1")

    # SMAs / EMAs
    sma10 = f"{ticker}_SMA_10"
    sma50 = f"{ticker}_SMA_50"
    ema20 = f"{ticker}_EMA_20"
    ema50 = f"{ticker}_EMA_50"
    macd_name = f"{ticker}_MACD"
    macd_sig = f"{ticker}_MACD_sig"
    macd_hist = f"{ticker}_MACD_hist"
    rsi14 = f"{ticker}_RSI_14"
    stochK = f"{ticker}_StochK"
    stochD = f"{ticker}_StochD"

    feats[sma10] = df[close].rolling(10).mean()
    feats[sma50] = df[close].rolling(50).mean()
    feats[ema20] = df[close].ewm(span=20).mean()
    feats[ema50] = df[close].ewm(span=50).mean()

    register_feature(registry, sma10, "shift_1")
    register_feature(registry, sma50, "shift_1")
    register_feature(registry, ema20, "shift_1")
    register_feature(registry, ema50, "shift_1")

    # MACD
    ema12 = df[close].ewm(span=12).mean()
    ema26 = df[close].ewm(span=26).mean()
    macd = ema12 - ema26

    feats[macd_name] = macd
    feats[macd_sig] = macd.ewm(span=9).mean()
    feats[macd_hist] = feats[macd_name] - feats[macd_sig]

    register_feature(registry, macd_name, "shift_1")
    register_feature(registry, macd_sig, "shift_1")
    register_feature(registry, macd_hist, "shift_1")

    # RSI (14)
    delta = df[close].diff()
    up = delta.clip(lower=0).rolling(14).mean()
    down = (-delta.clip(lower=0)).rolling(14).mean()
    rs = up / down
    feats[rsi14] = 100 - (100 / (1 + rs))

    register_feature(registry, rsi14, "shift_1")

    # Stochastics
    low14 = df[close].rolling(14).min()
    high14 = df[close].rolling(14).max()
    feats[stochK] = 100 * (df[close] - low14) / (high14 - low14)
    feats[stochD] = feats[stochK].rolling(3).mean()

    register_feature(registry, stochK, "shift_1")
    register_feature(registry, stochD, "shift_1")

    return feats


# ---------------------------------------------------------
# VOLATILITY FEATURES
# ---------------------------------------------------------
def add_vol_features(df: pd.DataFrame, ticker: str, open_: str, high: str, low: str, close: str, registry: dict) -> dict:
    feats = {}

    # Return-based vol
    name_vol20 = f"{ticker}_Vol_20"
    feats[name_vol20] = df[f"{ticker}_Return_1d"].rolling(20).std()
    register_feature(registry, name_vol20, "shift_1")

    # Parkinson volatility
    name_parkinson = f"{ticker}_Parkinson_20"
    feats[name_parkinson] = (
        (np.log(df[high] / df[low]) ** 2).rolling(20).mean()
        * (1 / (4 * np.log(2)))
    )
    register_feature(registry, name_parkinson, "shift_1")

    # Garman-Klass volatility
    name_gk = f"{ticker}_GK_20"
    feats[name_gk] = (
        0.5 * (np.log(df[high] / df[low]) ** 2)
        - (2 * np.log(np.e) - 1) * (np.log(df[close] / df[open_]) ** 2)
    ).rolling(20).mean()
    register_feature(registry, name_gk, "shift_1")

    # Bollinger width
    name_bb = f"{ticker}_BB_width"
    sma20 = df[close].rolling(20).mean()
    std20 = df[close].rolling(20).std()
    feats[name_bb] = (2 * std20) / sma20
    register_feature(registry, name_bb, "shift_1")

    return feats


# ---------------------------------------------------------
# VOLUME FEATURES
# ---------------------------------------------------------
def add_volume_features(df: pd.DataFrame, ticker: str, volume: str, close: str, registry: dict) -> dict:
    feats = {}

    roc = f"{ticker}_Volume_ROC"
    zname = f"{ticker}_Volume_Z"
    obv = f"{ticker}_OBV"

    feats[roc] = df[volume].pct_change(5)
    register_feature(registry, roc, "shift_1")

    feats[zname] = (df[volume] - df[volume].rolling(20).mean()) / df[volume].rolling(20).std()
    register_feature(registry, zname, "shift_1")

    feats[obv] = (np.sign(df[close].diff()) * df[volume]).fillna(0).cumsum()
    register_feature(registry, obv, "shift_1")

    return feats


# ---------------------------------------------------------
# ENTROPY FEATURE
# ---------------------------------------------------------
def add_entropy(df: pd.DataFrame, ticker: str, registry: dict) -> dict:
    feats = {}
    ret = f"{ticker}_Return_1d"

    if ret in df.columns:
        name = f"{ticker}_Entropy_20"
        feats[name] = (df[ret] ** 2).rolling(20).sum()
        register_feature(registry, name, "shift_1")

    return feats


In [8]:
# ======================================================================
# STEP D — CROSS-TICKER FEATURES RELATIVE TO THE TARGET TICKER
# ======================================================================
# These features compare each ticker to the target ticker's Close.
# Because they depend on *today’s* prices, all of them get shift_1.


def add_cross_features(df: pd.DataFrame, target: str, ticker: str, registry: dict) -> dict:
    """
    Create cross-ticker features between TARGET and TICKER.

    target : the ticker you are forecasting (e.g. "PPH")
    ticker : the other ticker to compare against

    Returns dict of new features.
    All features depend on OHLCV(t) → rule = shift_1
    """
    feats = {}

    target_col = f"{target}_Close"
    ticker_col = f"{ticker}_Close"

    # if either side is missing → no features
    if target_col not in df.columns or ticker_col not in df.columns:
        return feats

    # -------------------------
    # 1. Price Ratio
    # -------------------------
    ratio = f"{ticker}_Ratio_{target}"
    feats[ratio] = df[ticker_col] / df[target_col]
    register_feature(registry, ratio, "shift_1")

    # -------------------------
    # 2. Relative Strength (20d)
    # RS20 = return_ticker(20d) - return_target(20d)
    # -------------------------
    rs20 = f"{ticker}_RS_20"
    feats[rs20] = df[ticker_col].pct_change(20) - df[target_col].pct_change(20)
    register_feature(registry, rs20, "shift_1")

    # -------------------------
    # 3. Rolling Correlation (60d)
    # -------------------------
    corr = f"{ticker}_Corr_{target}_60"
    feats[corr] = df[ticker_col].rolling(60).corr(df[target_col])
    register_feature(registry, corr, "shift_1")

    return feats


In [9]:
# ======================================================================
# STEP E — PCA LATENT FACTORS (3 COMPONENTS)
# ======================================================================
# PCA is fit on returns(t), so PCA_1/2/3 must use shift_1.
# These features capture common sector/market structure.

from sklearn.decomposition import PCA


def add_pca_features(df: pd.DataFrame, tickers: list, registry: dict) -> dict:
    """
    Compute PCA latent factors from 1-day returns of all tickers.

    Returns a dict of:
        PCA_1, PCA_2, PCA_3

    All receive rule = "shift_1" because they use OHLCV(t).
    """
    feats = {}

    # Collect available return columns
    ret_cols = [
        f"{t}_Return_1d"
        for t in tickers
        if f"{t}_Return_1d" in df.columns
    ]

    # Need enough data to fit PCA robustly
    data = df[ret_cols].dropna()
    if len(data) < 200:     # minimum stability requirement
        return feats

    try:
        pca = PCA(n_components=3)
        comps = pca.fit_transform(data)

        # Create full-length series aligned to df index
        feats["PCA_1"] = pd.Series(comps[:, 0], index=data.index)
        feats["PCA_2"] = pd.Series(comps[:, 1], index=data.index)
        feats["PCA_3"] = pd.Series(comps[:, 2], index=data.index)

        # Register shift rules
        register_feature(registry, "PCA_1", "shift_1")
        register_feature(registry, "PCA_2", "shift_1")
        register_feature(registry, "PCA_3", "shift_1")

    except Exception as e:
        print(f"PCA failed: {e}")

    return feats


In [10]:
# ======================================================================
# STEP F — CALENDAR & MACRO EVENT FEATURES (registry-aware)
# ======================================================================
# These features do NOT depend on OHLCV(t).
# Safe rules:
#   - "no_shift"        → calendar features known before market open
#   - "shift_plus_k"    → future macro events (CPI, NFP, FOMC, PPI, GDP)
#
# These rules allow the shift engine to later build:
#   is_cpi_day_t+1, is_cpi_day_t+2, ..., is_cpi_day_t+horizon
# ======================================================================

import pandas as pd
import holidays
from pandas_market_calendars import get_calendar


# -----------------------------------------------------------
# FOMC Dates
# -----------------------------------------------------------
def fetch_fomc_dates():
    """
    Fetch FOMC meeting dates from Federal Reserve website.
    Returns a Pandas Series for .dt.isocalendar().
    """
    url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
    try:
        tables = pd.read_html(url)
        dates = pd.to_datetime(tables[0].iloc[:, 0], errors="coerce").dropna()
        return pd.Series(dates)
    except:
        fallback = pd.to_datetime([
            "2024-01-31","2024-03-20","2024-05-01",
            "2024-06-12","2024-07-31","2024-09-18",
            "2024-11-07","2024-12-18"
        ])
        return pd.Series(fallback)


# -----------------------------------------------------------
# Macro release dates (user-supplied)
# -----------------------------------------------------------
def macro_calendar():
    """Returns dict of event_name → Series of event dates."""
    events = {
        "cpi": ["2024-01-11","2024-02-13","2024-03-12","2024-04-10"],
        "nfp": ["2024-01-05","2024-02-02","2024-03-08","2024-04-05"],
        "ppi": ["2024-01-12","2024-02-16","2024-03-14","2024-04-11"],
        "gdp": ["2024-01-25","2024-02-28","2024-03-28","2024-04-25"],
    }
    return {k: pd.Series(pd.to_datetime(v)) for k, v in events.items()}


# -----------------------------------------------------------
# Calendar basics
# -----------------------------------------------------------
def add_calendar_basics(df, registry):
    df["day_of_week"]    = df["date"].dt.dayofweek
    df["day_of_month"]   = df["date"].dt.day
    df["month"]          = df["date"].dt.month
    df["quarter"]        = df["date"].dt.quarter
    df["is_month_end"]   = df["date"].dt.is_month_end.astype(int)
    df["is_month_start"] = df["date"].dt.is_month_start.astype(int)
    df["is_year_end"]    = ((df["month"] == 12) & (df["day_of_month"] == 31)).astype(int)

    for col in [
        "day_of_week", "day_of_month", "month", "quarter",
        "is_month_end", "is_month_start", "is_year_end"
    ]:
        register_feature(registry, col, "no_shift")

    return df


# -----------------------------------------------------------
# US holiday adjacency
# -----------------------------------------------------------
def add_holiday_features(df, registry):
    us_holidays = holidays.US()
    df["is_holiday_adjacent"] = df["date"].apply(
        lambda d: int((d + pd.Timedelta(days=1) in us_holidays) or
                      (d - pd.Timedelta(days=1) in us_holidays))
    )

    register_feature(registry, "is_holiday_adjacent", "no_shift")
    return df


# -----------------------------------------------------------
# OPEX week
# -----------------------------------------------------------
def add_opex_features(df, registry):
    df["is_opex_week"] = df["date"].apply(
        lambda d: int((d.weekday() == 4) and (15 <= d.day <= 21))
    )

    register_feature(registry, "is_opex_week", "no_shift")
    return df


# -----------------------------------------------------------
# FOMC week (volatility clusters)
# -----------------------------------------------------------
def add_fomc_features(df, registry):
    fomc_dates = fetch_fomc_dates()
    fomc_weeks = fomc_dates.dt.isocalendar().week.unique()

    df["is_fomc_week"] = df["date"].dt.isocalendar().week.isin(fomc_weeks).astype(int)
    register_feature(registry, "is_fomc_week", "no_shift")
    return df


# -----------------------------------------------------------
# High-impact macroeconomic event flags
# -----------------------------------------------------------
def add_macro_features(df, registry):
    macros = macro_calendar()

    for name, dates in macros.items():

        # Day-level event
        day_col = f"is_{name}_day"
        df[day_col] = df["date"].isin(dates).astype(int)
        register_feature(registry, day_col, "shift_plus_k")

        # Event-week indicator – future relevance ambiguous
        week_col = f"is_{name}_week"
        df[week_col] = df["date"].dt.isocalendar().week.isin(
            dates.dt.isocalendar().week.unique()
        ).astype(int)
        register_feature(registry, week_col, "shift_plus_k")

    return df


# -----------------------------------------------------------
# Master wrapper — assumes df.index is datetime index
# -----------------------------------------------------------
def add_all_calendar_features(df, registry):
    """
    Adds calendar, holiday, OPEX, FOMC, and macro-event features.
    Applies only registry tagging; no shifts are applied here.
    """
    df = df.copy()

    # Convert index → date column
    df = df.reset_index().rename(columns={df.index.name or "index": "date"})
    df["date"] = pd.to_datetime(df["date"])

    df = add_calendar_basics(df, registry)
    df = add_holiday_features(df, registry)
    df = add_opex_features(df, registry)
    df = add_fomc_features(df, registry)
    df = add_macro_features(df, registry)

    return df


In [11]:
# ======================================================================
# STEP G — SHIFT ENGINE (APPLIES ALL SHIFT RULES)
# ======================================================================
# Interprets the registry and applies:
#   - shift_1        → df[col] = df[col].shift(1)
#   - no_shift       → leave unchanged
#   - shift_plus_k   → create k=1…horizon shifted columns
#
# Ensures:
#   - ZERO data leakage
#   - multi-step recursive forecasting support
# ======================================================================


def apply_shift_rules(df: pd.DataFrame, registry: dict, horizon: int) -> pd.DataFrame:
    """
    Applies temporal alignment to all features according to the registry.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe containing all raw features.
    registry : dict
        feature_name → rule
        where rule ∈ {"shift_1", "no_shift", "shift_plus_k"}
    horizon : int
        Forecast horizon (e.g., 30, 60, etc.)

    Returns
    -------
    pd.DataFrame
        A new dataframe with aligned features.
    """
    df = df.copy()
    new_cols = {}

    for col, rule in registry.items():

        # ---------------------------
        # RULE 1 — shift_1
        # ---------------------------
        if rule == "shift_1":
            new_cols[col] = df[col].shift(1)

        # ---------------------------
        # RULE 2 — no_shift
        # ---------------------------
        elif rule == "no_shift":
            new_cols[col] = df[col]   # unchanged

        # ---------------------------
        # RULE 3 — shift_plus_k
        # ---------------------------
        elif rule == "shift_plus_k":
            # Create ladder of k = 1…horizon
            for k in range(1, horizon + 1):
                new_name = f"{col}_t+{k}"
                new_cols[new_name] = df[col].shift(k)

            # We do NOT keep the original (unshifted) event flag
            # because it's meaningless in recursive forecasting.
            continue

        else:
            raise ValueError(f"Unknown shift rule '{rule}' for column '{col}'")

    # ---------------------------
    # Merge all new features
    # ---------------------------
    out = pd.concat([df, pd.DataFrame(new_cols, index=df.index)], axis=1)

    return out


In [12]:
# ======================================================================
# STEP H — GENERATE MARKDOWN DOCUMENTATION
# ======================================================================
# Creates a markdown file listing:
#   feature_name
#   transformation_rule
#   resulting feature names (including expanded future-event ladders)
#
# This does NOT affect the data pipeline — it is documentation only.
# ======================================================================


def generate_markdown_from_registry(registry: dict, horizon: int, path: str):
    """
    Generate a markdown documentation table from the feature registry.

    Parameters
    ----------
    registry : dict
        feature_name → rule
    horizon : int
        Forecast horizon (determines how many shift_plus_k features to list)
    path : str
        Output markdown file path

    Returns
    -------
    str : markdown text (also saved to 'path')
    """

    lines = []
    lines.append("# Feature Transformation Table\n")
    lines.append("This table documents how each feature is temporally aligned.\n")
    lines.append("\n")
    lines.append("| Feature Name | Rule | Resulting Columns |\n")
    lines.append("|--------------|------|------------------|\n")

    for col, rule in registry.items():

        # ---------------------------------------
        # 1. shift_1  → one resulting column
        # ---------------------------------------
        if rule == "shift_1":
            resulting = f"{col} (t-1)"

        # ---------------------------------------
        # 2. no_shift → one resulting column
        # ---------------------------------------
        elif rule == "no_shift":
            resulting = col

        # ---------------------------------------
        # 3. shift_plus_k → many resulting columns
        # ---------------------------------------
        elif rule == "shift_plus_k":
            resulting = ", ".join(
                [f"{col}_t+{k}" for k in range(1, horizon + 1)]
            )

        else:
            resulting = "UNKNOWN RULE"

        # add markdown row
        lines.append(f"| {col} | {rule} | {resulting} |\n")

    # Write markdown file
    md = "".join(lines)
    with open(path, "w") as f:
        f.write(md)

    return md


In [2]:
# Find start dates of each ETF
start_dates = {col: df[col].first_valid_index() for col in df.columns if col.endswith("_Close")}
print(start_dates)

{'VIXY_Close': Timestamp('2011-01-04 00:00:00'), 'XPH_Close': Timestamp('2011-01-04 00:00:00'), 'VHT_Close': Timestamp('2011-01-04 00:00:00'), 'IHE_Close': Timestamp('2011-01-04 00:00:00'), 'XBI_Close': Timestamp('2011-01-04 00:00:00'), 'XLV_Close': Timestamp('2011-01-04 00:00:00'), 'SPY_Close': Timestamp('2011-01-04 00:00:00'), 'PPH_Close': Timestamp('2011-01-04 00:00:00'), 'IBB_Close': Timestamp('2011-01-04 00:00:00')}


In [3]:
# Find start date to use when all ETFS have data
global_start = max(start_dates.values())
df = df.loc[df.index >= global_start]

In [4]:
df.head(5)

,VIXY_Open,VIXY_High,VIXY_Low,VIXY_Close,VIXY_Volume,XPH_Open,XPH_High,XPH_Low,XPH_Close,XPH_Volume,...,PPH_Open,PPH_High,PPH_Low,PPH_Close,PPH_Volume,IBB_Open,IBB_High,IBB_Low,IBB_Close,IBB_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-04,633120.0,650800.0,633120.0,633840.0,14,23.285000,23.285000,22.965000,23.059999,42000,...,32.794998,32.965000,32.695000,32.965000,529200,31.620001,31.620001,31.139999,31.299999,1088100
2011-01-05,640400.0,640800.0,617440.0,620400.0,9,22.945000,23.250000,22.945000,23.219999,54400,...,32.840000,33.060001,32.810001,33.005001,1349600,31.299999,31.623333,31.240000,31.549999,1756800
2011-01-06,619520.0,629440.0,614960.0,623040.0,10,23.360001,23.395000,23.209999,23.350000,34600,...,33.014999,33.264999,33.000000,33.259998,363000,31.450001,31.750000,31.450001,31.696667,714300
2011-01-07,617920.0,640080.0,609440.0,624320.0,5,23.290001,23.400000,23.115000,23.375000,96200,...,33.220001,33.320000,33.084999,33.275002,391000,31.673332,31.796667,31.466667,31.683332,1017300
2011-01-10,637120.0,646960.0,622080.0,623040.0,9,23.370001,23.504999,23.250000,23.480000,59600,...,33.154999,33.220001,33.055000,33.115002,302800,31.660000,31.693333,31.440001,31.646667,938400


In [18]:
df.index

DatetimeIndex(['2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
               '2011-01-10', '2011-01-11', '2011-01-12', '2011-01-13',
               '2011-01-14', '2011-01-18',
               ...
               '2025-11-14', '2025-11-17', '2025-11-18', '2025-11-19',
               '2025-11-20', '2025-11-21', '2025-11-24', '2025-11-25',
               '2025-11-26', '2025-11-28'],
              dtype='datetime64[ns]', name='Date', length=3749, freq=None)

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from hmmlearn.hmm import GaussianHMM
from tqdm.auto import tqdm
import logging
from scipy.stats import skew, kurtosis


# ----------------------------------------------------------
# Logging
# ----------------------------------------------------------
logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger(__name__)


# ==========================================================
#  UTILITY HELPERS
# ==========================================================

def infer_tickers(df: pd.DataFrame):
    """Infer tickers from *_Close columns."""
    return sorted({c.split("_")[0] for c in df.columns if c.endswith("_Close")})


def ohlcv_cols(df: pd.DataFrame, ticker: str):
    """Return Open/High/Low/Close/Volume column names if present."""
    cols = [f"{ticker}_{c}" for c in ["Open", "High", "Low", "Close", "Volume"]]
    return cols if all(c in df.columns for c in cols) else None


# ----------------------------------------------------------
# Rolling R² for trend quality
# ----------------------------------------------------------
def rolling_r2(series: pd.Series, window: int) -> pd.Series:
    r2 = np.full(len(series), np.nan)
    t = np.arange(window)

    for i in range(window, len(series)):
        y = series.iloc[i-window:i]
        if y.isna().any():
            continue

        coef = np.polyfit(t, y, 1)
        y_pred = coef[0] * t + coef[1]

        ss_res = ((y - y_pred) ** 2).sum()
        ss_tot = ((y - y.mean()) ** 2).sum()

        if ss_tot != 0:
            r2[i] = 1 - ss_res / ss_tot

    return pd.Series(r2, index=series.index)


# ----------------------------------------------------------
# Rolling Beta
# ----------------------------------------------------------
def rolling_beta(ret_a: pd.Series, ret_b: pd.Series, window: int):
    cov = ret_a.rolling(window).cov(ret_b)
    var = ret_b.rolling(window).var()
    return cov / var


# ----------------------------------------------------------
# ADX wrapper
# ----------------------------------------------------------
def ADX(high, low, close, window=14):
    import ta
    return ta.trend.adx(high, low, close, window)


# ==========================================================
# PER-TICKER BASE TECHNICAL FEATURES
# ==========================================================

def add_momentum_features(df, ticker, close):
    feats = {}

    # Returns
    for w in [1, 5, 10, 20]:
        feats[f"{ticker}_Return_{w}d"] = df[close].pct_change(w)

    # SMAs / EMAs
    feats[f"{ticker}_SMA_10"] = df[close].rolling(10).mean()
    feats[f"{ticker}_SMA_50"] = df[close].rolling(50).mean()
    feats[f"{ticker}_EMA_20"] = df[close].ewm(span=20).mean()
    feats[f"{ticker}_EMA_50"] = df[close].ewm(span=50).mean()
    feats[f"{ticker}_MA_Cross"] = feats[f"{ticker}_EMA_20"] / feats[f"{ticker}_EMA_50"]

    # MACD
    ema12 = df[close].ewm(span=12).mean()
    ema26 = df[close].ewm(span=26).mean()
    macd = ema12 - ema26
    feats[f"{ticker}_MACD"] = macd
    feats[f"{ticker}_MACD_sig"] = macd.ewm(span=9).mean()
    feats[f"{ticker}_MACD_hist"] = feats[f"{ticker}_MACD"] - feats[f"{ticker}_MACD_sig"]

    # RSI
    delta = df[close].diff()
    up = delta.clip(lower=0).rolling(14).mean()
    down = (-delta.clip(lower=0)).rolling(14).mean()
    rs = up / down
    feats[f"{ticker}_RSI_14"] = 100 - (100 / (1 + rs))

    # Stochastics
    low14 = df[close].rolling(14).min()
    high14 = df[close].rolling(14).max()
    feats[f"{ticker}_StochK"] = 100 * (df[close] - low14) / (high14 - low14)
    feats[f"{ticker}_StochD"] = feats[f"{ticker}_StochK"].rolling(3).mean()

    return feats


def add_vol_features(df, ticker, open_, high, low, close):
    feats = {}

    feats[f"{ticker}_Vol_20"] = df[f"{ticker}_Return_1d"].rolling(20).std()

    feats[f"{ticker}_Parkinson_20"] = (
        (np.log(df[high] / df[low]) ** 2).rolling(20).mean()
        * (1 / (4 * np.log(2)))
    )

    feats[f"{ticker}_GK_20"] = (
        0.5 * (np.log(df[high] / df[low]) ** 2)
        - (2 * np.log(np.e) - 1) * (np.log(df[close] / df[open_]) ** 2)
    ).rolling(20).mean()

    sma20 = df[close].rolling(20).mean()
    std20 = df[close].rolling(20).std()
    feats[f"{ticker}_BB_width"] = (2 * std20) / sma20

    return feats


def add_volume_features(df, ticker, volume, close):
    feats = {}

    feats[f"{ticker}_Volume_ROC"] = df[volume].pct_change(5)
    feats[f"{ticker}_Volume_Z"]   = (df[volume] - df[volume].rolling(20).mean()) / df[volume].rolling(20).std()
    feats[f"{ticker}_OBV"]        = (np.sign(df[close].diff()) * df[volume]).fillna(0).cumsum()

    return feats


def add_entropy(df, ticker):
    feats = {}
    ret = f"{ticker}_Return_1d"
    if ret in df.columns:
        feats[f"{ticker}_Entropy_20"] = (df[ret] ** 2).rolling(20).sum()
    return feats


def add_hmm_states(df, ticker):
    feats = {}
    ret = f"{ticker}_Return_1d"
    if ret not in df.columns:
        return feats

    series = df[ret].dropna()
    if len(series) < 200:
        return feats

    hmm = GaussianHMM(
        n_components=3,
        covariance_type="full",
        n_iter=800,
        tol=1e-3,
        init_params="mct",
        params="mct"
    )

    try:
        hmm.fit(series.values.reshape(-1, 1))
        feats[f"{ticker}_HMM"] = pd.Series(hmm.predict(series.values.reshape(1, -1)[0]), index=series.index)

    except Exception as e:
        logger.warning(f"HMM failed for {ticker}: {e}")

    return feats


# ==========================================================
# ADVANCED PER-TICKER FEATURES (beta, skew, vov, ADX, trend R²)
# ==========================================================

def add_advanced_ticker_features(df, ticker, tickers):
    feats = {}

    close = df[f"{ticker}_Close"]
    high  = df[f"{ticker}_High"]
    low   = df[f"{ticker}_Low"]
    vol   = df[f"{ticker}_Volume"]
    ret1  = df[f"{ticker}_Return_1d"]

    # ---------------------------------------
    # 1. LAGGED CLOSE
    # ---------------------------------------
    feats[f"{ticker}_Close_lag1"] = close.shift(1)

    # ---------------------------------------
    # 2. Betas + residuals vs all other tickers
    # ---------------------------------------
    for bench in tickers:
        if bench == ticker:
            continue
        if f"{bench}_Return_1d" not in df.columns:
            continue

        bench_ret = df[f"{bench}_Return_1d"]

        for w in [20, 60]:
            beta = rolling_beta(ret1, bench_ret, w)
            feats[f"{ticker}_beta_{bench}_{w}"] = beta
            feats[f"{ticker}_residual_{bench}_{w}"] = ret1 - beta * bench_ret

    # ---------------------------------------
    # 3. Skew & Kurtosis
    # ---------------------------------------
    for w in [20, 60]:
        feats[f"{ticker}_skew_{w}"] = ret1.rolling(w).apply(
            lambda x: skew(x, bias=False) if not np.isnan(x).any() else np.nan,
            raw=False
        )
        feats[f"{ticker}_kurt_{w}"] = ret1.rolling(w).apply(
            lambda x: kurtosis(x, bias=False) if not np.isnan(x).any() else np.nan,
            raw=False
        )

    # ---------------------------------------
    # 4. Vol-of-Vol
    # ---------------------------------------
    vol20 = ret1.rolling(20).std()
    for w in [20, 60]:
        feats[f"{ticker}_vov_{w}"] = vol20.diff().rolling(w).std()

    # ---------------------------------------
    # 5. Trend R²
    # ---------------------------------------
    feats[f"{ticker}_trend_r2_20"] = rolling_r2(close, 20)
    feats[f"{ticker}_trend_r2_60"] = rolling_r2(close, 60)

    # ---------------------------------------
    # 6. ADX
    # ---------------------------------------
    feats[f"{ticker}_adx_14"] = ADX(high, low, close, 14)

    # ---------------------------------------
    # 7. Volume Microstructure
    # ---------------------------------------
    feats[f"{ticker}_volume_percentile_252"] = (
        vol.rolling(252).apply(
            lambda x: (pd.Series(x).rank().iloc[-1]) / 252 if len(x) == 252 else np.nan,
            raw=False
        )
    )
    feats[f"{ticker}_volume_trend_20"] = rolling_r2(vol, 20)
    feats[f"{ticker}_volume_dryup_flag"] = vol < vol.rolling(60).mean() * 0.5

    return feats


# ==========================================================
# CROSS-TICKER FEATURES
# ==========================================================

def add_cross_features(df, tickers):
    feats = {}
    for a in tickers:
        for b in tickers:
            if a == b:
                continue
            A = f"{a}_Close"
            B = f"{b}_Close"

            if A not in df.columns or B not in df.columns:
                continue

            feats[f"{a}_Ratio_{b}"] = df[A] / df[B]
            feats[f"{a}_RS_20_{b}"] = df[A].pct_change(20) - df[B].pct_change(20)
            feats[f"{a}_Corr60_{b}"] = df[A].rolling(60).corr(df[B])

    return feats


# ==========================================================
# PCA
# ==========================================================

def add_pca(df, tickers):
    feats = {}
    ret_cols = [f"{t}_Return_1d" for t in tickers if f"{t}_Return_1d" in df.columns]

    X = df[ret_cols].dropna()
    if len(X) < 200 or len(ret_cols) < 3:
        return feats

    try:
        pca = PCA(n_components=3)
        comps = pca.fit_transform(X)

        feats["PCA_1"] = pd.Series(comps[:, 0], index=X.index)
        feats["PCA_2"] = pd.Series(comps[:, 1], index=X.index)
        feats["PCA_3"] = pd.Series(comps[:, 2], index=X.index)

    except Exception as e:
        logger.warning(f"PCA failed: {e}")

    return feats


# ==========================================================
#  MASTER PIPELINE (TICKER-NEUTRAL)
# ==========================================================

def create_all_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fully unified ticker-neutral feature engine.

    Order:
    1. Technical per-ticker features
    2. Volatility features
    3. Volume features
    4. Entropy
    5. HMM states
    6. Advanced per-ticker features (beta, skew, ADX, trend R², vov)
    7. Cross-ticker features
    8. PCA latent factors
    9. Single concat at end

    Returns:
        df with all new features added.
    """
    df = df.copy()
    tickers = infer_tickers(df)

    logger.info(f"Building features for {len(tickers)} tickers...")

    new_feats = {}

    # ------------------ Per-ticker blocks ------------------
    for t in tqdm(tickers, desc="Per-ticker base features"):
        cols = ohlcv_cols(df, t)
        if not cols:
            continue

        open_, high, low, close, volume = cols

        new_feats.update(add_momentum_features(df, t, close))
        new_feats.update(add_vol_features(df, t, open_, high, low, close))
        new_feats.update(add_volume_features(df, t, volume, close))
        new_feats.update(add_entropy(df, t))
        new_feats.update(add_hmm_states(df, t))

    # ------------------ Advanced per-ticker blocks ------------------
    for t in tqdm(tickers, desc="Per-ticker advanced features"):
        new_feats.update(add_advanced_ticker_features(df, t, tickers))

    # ------------------ Cross-ticker ------------------
    logger.info("Building cross-ticker features...")
    new_feats.update(add_cross_features(df, tickers))

    # ------------------ PCA ------------------
    logger.info("Computing PCA...")
    new_feats.update(add_pca(df, tickers))

    # ------------------ Final Merge ------------------
    logger.info("Merging feature matrix...")
    df = pd.concat([df, pd.DataFrame(new_feats, index=df.index)], axis=1)

    logger.info("Feature engineering complete.")
    return df


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from hmmlearn.hmm import GaussianHMM
from tqdm.auto import tqdm
import logging

# ---------------------------------------------------------------------
# Logging setup (clean output, no spam)
# ---------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger(__name__)


# =====================================================================
#  HELPER FUNCTIONS (all return dicts to avoid fragmentation)
# =====================================================================

def ohlcv_cols(df: pd.DataFrame, ticker: str):
    """
    Returns (Open, High, Low, Close, Volume) column names for a ticker.
    If any are missing → return None.
    """
    cols = [f"{ticker}_{c}" for c in ["Open", "High", "Low", "Close", "Volume"]]
    return cols if all(c in df.columns for c in cols) else None


# ----------------------------- MOMENTUM ------------------------------
def add_momentum_features(df: pd.DataFrame, ticker: str, close: str) -> dict:
    feats = {}

    # Simple returns
    for w in [1, 5, 10, 20]:
        feats[f"{ticker}_Return_{w}d"] = df[close].pct_change(w)

    # SMAs / EMAs
    feats[f"{ticker}_SMA_10"] = df[close].rolling(10).mean()
    feats[f"{ticker}_SMA_50"] = df[close].rolling(50).mean()
    feats[f"{ticker}_EMA_20"] = df[close].ewm(span=20).mean()
    feats[f"{ticker}_EMA_50"] = df[close].ewm(span=50).mean()
    feats[f"{ticker}_MA_Cross"] = feats[f"{ticker}_EMA_20"] / feats[f"{ticker}_EMA_50"]

    # MACD
    ema12 = df[close].ewm(span=12).mean()
    ema26 = df[close].ewm(span=26).mean()
    macd = ema12 - ema26

    feats[f"{ticker}_MACD"] = macd
    feats[f"{ticker}_MACD_sig"] = macd.ewm(span=9).mean()
    feats[f"{ticker}_MACD_hist"] = feats[f"{ticker}_MACD"] - feats[f"{ticker}_MACD_sig"]

    # RSI
    delta = df[close].diff()
    up = delta.clip(lower=0).rolling(14).mean()
    down = (-delta.clip(lower=0)).rolling(14).mean()
    rs = up / down
    feats[f"{ticker}_RSI_14"] = 100 - (100 / (1 + rs))

    # Stochastics
    low14 = df[close].rolling(14).min()
    high14 = df[close].rolling(14).max()
    feats[f"{ticker}_StochK"] = 100 * (df[close] - low14) / (high14 - low14)
    feats[f"{ticker}_StochD"] = feats[f"{ticker}_StochK"].rolling(3).mean()

    return feats


# ----------------------------- VOLATILITY ----------------------------
def add_vol_features(
    df: pd.DataFrame,
    ticker: str,
    open_: str,
    high: str,
    low: str,
    close: str
) -> dict:
    feats = {}

    # Standard deviation of returns
    feats[f"{ticker}_Vol_20"] = df[f"{ticker}_Return_1d"].rolling(20).std()

    # Parkinson volatility
    feats[f"{ticker}_Parkinson_20"] = (
        (np.log(df[high] / df[low]) ** 2).rolling(20).mean()
        * (1 / (4 * np.log(2)))
    )

    # Garman-Klass volatility
    feats[f"{ticker}_GK_20"] = (
        0.5 * (np.log(df[high] / df[low]) ** 2)
        - (2 * np.log(np.e) - 1) * (np.log(df[close] / df[open_]) ** 2)
    ).rolling(20).mean()

    # Bollinger width
    sma20 = df[close].rolling(20).mean()
    std20 = df[close].rolling(20).std()
    feats[f"{ticker}_BB_width"] = (2 * std20) / sma20

    return feats


# ----------------------------- VOLUME -------------------------------
def add_volume_features(df: pd.DataFrame, ticker: str, volume: str, close: str) -> dict:
    feats = {}

    feats[f"{ticker}_Volume_ROC"] = df[volume].pct_change(5)

    feats[f"{ticker}_Volume_Z"] = (
        df[volume] - df[volume].rolling(20).mean()
    ) / df[volume].rolling(20).std()

    feats[f"{ticker}_OBV"] = (np.sign(df[close].diff()) * df[volume]).fillna(0).cumsum()

    return feats


# ----------------------------- ENTROPY ------------------------------
def add_entropy(df: pd.DataFrame, ticker: str) -> dict:
    feats = {}
    ret = f"{ticker}_Return_1d"
    if ret in df.columns:
        feats[f"{ticker}_Entropy_20"] = (df[ret] ** 2).rolling(20).sum()
    return feats


# ------------------------------ HMM --------------------------------
def add_hmm_states(df: pd.DataFrame, ticker: str) -> dict:
    feats = {}
    ret = f"{ticker}_Return_1d"
    if ret not in df.columns:
        return feats

    series = df[ret].dropna()
    if len(series) < 200:
        return feats

    # More stable HMM configuration
    hmm = GaussianHMM(
        n_components=3,
        covariance_type="full",
        n_iter=800,        # higher iteration cap
        tol=1e-3,          # easier to satisfy convergence
        init_params="mct", # more stable initialization
        params="mct"       # no need to estimate weights
    )

    try:
        # Fit & predict — warnings are NOT suppressed
        hmm.fit(series.values.reshape(-1, 1))
        states = hmm.predict(series.values.reshape(-1, 1))

        feats[f"{ticker}_HMM"] = pd.Series(states, index=series.index)

    except Exception as e:
        logger.warning(f"HMM failed for {ticker}: {e}")

    return feats


# ----------------------------- CROSS-TICKER -------------------------
def add_cross_features(df: pd.DataFrame, base_ticker: str, target_ticker: str) -> dict:
    feats = {}
    base = f"{base_ticker}_Close"
    tgt  = f"{target_ticker}_Close"

    if base not in df.columns or tgt not in df.columns:
        return feats

    feats[f"{target_ticker}_Ratio_{base_ticker}"] = df[tgt] / df[base]
    feats[f"{target_ticker}_RS_20"] = df[tgt].pct_change(20) - df[base].pct_change(20)
    feats[f"{target_ticker}_Corr_{base_ticker}_60"] = df[tgt].rolling(60).corr(df[base])

    return feats


# ----------------------------- PCA --------------------------------
def add_pca(df: pd.DataFrame, tickers: list) -> dict:
    feats = {}

    ret_cols = [f"{t}_Return_1d" for t in tickers if f"{t}_Return_1d" in df.columns]
    data = df[ret_cols].dropna()

    if len(data) < 200:
        return feats

    try:
        pca = PCA(n_components=3)
        comps = pca.fit_transform(data)

        feats["PCA_1"] = pd.Series(comps[:, 0], index=data.index)
        feats["PCA_2"] = pd.Series(comps[:, 1], index=data.index)
        feats["PCA_3"] = pd.Series(comps[:, 2], index=data.index)

    except Exception as e:
        logger.warning(f"PCA failed: {e}")

    return feats



# =====================================================================
#  MAIN PIPELINE — CLEAN, SAFE, FAST
# =====================================================================

def create_all_features(df: pd.DataFrame, base: str = "PPH") -> pd.DataFrame:
    """
    Master feature engineering pipeline.

    - Fully leakage-safe
    - Fragmentation-free (only one concat)
    - Includes technicals, volatility, volume, entropy, HMM, PCA, cross-ticker.
    - Logs progress
    - Uses tqdm for progress bars
    """

    new_features = {}
    tickers = sorted({c.split('_')[0] for c in df.columns})

    logger.info(f"Building features for {len(tickers)} tickers...")
    pbar = tqdm(tickers, desc="Per-ticker features")

    # -------------------- Per-ticker features --------------------
    for ticker in pbar:
        cols = ohlcv_cols(df, ticker)
        if not cols:
            continue

        open_, high, low, close, volume = cols

        new_features.update(add_momentum_features(df, ticker, close))
        new_features.update(add_vol_features(df, ticker, open_, high, low, close))
        new_features.update(add_volume_features(df, ticker, volume, close))
        new_features.update(add_entropy(df, ticker))
        new_features.update(add_hmm_states(df, ticker))

    # -------------------- Cross-ticker features -------------------
    logger.info("Building cross-ticker features...")
    for t in tqdm(tickers, desc="Cross features"):
        new_features.update(add_cross_features(df, base, t))

    # -------------------- PCA latent factors ----------------------
    logger.info("Fitting PCA on returns...")
    new_features.update(add_pca(df, tickers))

    # -------------------- CONCAT ONCE (NO FRAGMENTATION) ---------
    logger.info("Merging feature matrix...")
    df = pd.concat([df, pd.DataFrame(new_features, index=df.index)], axis=1)

    logger.info("Feature engineering complete.")
    return df

In [ ]:
df_full = create_all_features(data)
df_full.columns.to_list()

In [37]:
!pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [29]:
!pip install hmmlearn

In [31]:
df.head(5)

,VIXY_Open,VIXY_High,VIXY_Low,VIXY_Close,VIXY_Volume,XPH_Open,XPH_High,XPH_Low,XPH_Close,XPH_Volume,...,IBB_Open,IBB_High,IBB_Low,IBB_Close,IBB_Volume,IHE_Open,IHE_High,IHE_Low,IHE_Close,IHE_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-04,633120.0,650800.0,633120.0,633840.0,14,23.285000,23.285000,22.965000,23.059999,42000,...,31.620001,31.620001,31.139999,31.299999,1088100,21.650000,21.650000,21.370001,21.469999,54300
2011-01-05,640400.0,640800.0,617440.0,620400.0,9,22.945000,23.250000,22.945000,23.219999,54400,...,31.299999,31.623333,31.240000,31.549999,1756800,21.353333,21.576668,21.353333,21.563334,34200
2011-01-06,619520.0,629440.0,614960.0,623040.0,10,23.360001,23.395000,23.209999,23.350000,34600,...,31.450001,31.750000,31.450001,31.696667,714300,21.663334,21.663334,21.543333,21.626667,195300
2011-01-07,617920.0,640080.0,609440.0,624320.0,5,23.290001,23.400000,23.115000,23.375000,96200,...,31.673332,31.796667,31.466667,31.683332,1017300,21.656668,21.666668,21.466667,21.626667,21300
2011-01-10,637120.0,646960.0,622080.0,623040.0,9,23.370001,23.504999,23.250000,23.480000,59600,...,31.660000,31.693333,31.440001,31.646667,938400,21.530001,21.616667,21.480000,21.616667,46200


In [41]:
data = create_all_features(df, base="PPH")

Building features for 10 tickers...


Per-ticker features:   0%|          | 0/10 [00:00<?, ?it/s]

Model is not converging.  Current: 10698.493008110583 is not greater than 10698.52922953993. Delta is -0.0362214293472789
Model is not converging.  Current: 12243.895646050472 is not greater than 12243.901178246677. Delta is -0.005532196204512729
Model is not converging.  Current: 12385.812674391645 is not greater than 12385.818225453213. Delta is -0.005551061567530269
Model is not converging.  Current: 12228.998505180101 is not greater than 12229.012395682737. Delta is -0.013890502636058955
Model is not converging.  Current: 12329.586963596239 is not greater than 12329.5913701717. Delta is -0.0044065754609619034
Building cross-ticker features...


Cross features:   0%|          | 0/10 [00:00<?, ?it/s]

Fitting PCA on returns...
Merging feature matrix...
Feature engineering complete.


In [43]:
data.shape

(3749, 537)

In [44]:
!pip install holidays pandas-market-calendars requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.9 MB/s  0:00:00s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [pandas-market-calendars]4/6 [exchange-calendars]


In [49]:
import pandas as pd
import holidays
from pandas_market_calendars import get_calendar

# -----------------------------------------------------------
# Fetch Federal Reserve meeting dates (auto-updating if possible)
# -----------------------------------------------------------
def fetch_fomc_dates():
    """
    Fetch FOMC meeting dates directly from the Federal Reserve.
    Falls back to known dates if parsing fails.
    Returns a Pandas Series (not Index) for .dt access.
    """
    url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
    try:
        tables = pd.read_html(url)
        dates = pd.to_datetime(tables[0].iloc[:, 0], errors="coerce").dropna()
        return pd.Series(dates)     # <- FIX: return Series, not DatetimeIndex
    except:
        fallback = pd.to_datetime([
            "2024-01-31","2024-03-20","2024-05-01",
            "2024-06-12","2024-07-31","2024-09-18",
            "2024-11-07","2024-12-18"
        ])
        return pd.Series(fallback)  # <- FIX: return Series

# -----------------------------------------------------------
# Macro release calendar (CPI, NFP, PPI, GDP)
# -----------------------------------------------------------
def macro_calendar():
    """
    High-impact macroeconomic dates (extendable).
    Returns dict of name -> Series of dates.
    """
    events = {
        "cpi": ["2024-01-11","2024-02-13","2024-03-12","2024-04-10"],
        "nfp": ["2024-01-05","2024-02-02","2024-03-08","2024-04-05"],
        "ppi": ["2024-01-12","2024-02-16","2024-03-14","2024-04-11"],
        "gdp": ["2024-01-25","2024-02-28","2024-03-28","2024-04-25"],
    }
    return {k: pd.Series(pd.to_datetime(v)) for k, v in events.items()}

# -----------------------------------------------------------
# Basic calendar seasonality (strong signals)
# -----------------------------------------------------------
def add_calendar_basics(df):
    df["day_of_week"]    = df["date"].dt.dayofweek
    df["day_of_month"]   = df["date"].dt.day
    df["month"]          = df["date"].dt.month
    df["quarter"]        = df["date"].dt.quarter
    df["is_month_end"]   = df["date"].dt.is_month_end.astype(int)
    df["is_month_start"] = df["date"].dt.is_month_start.astype(int)
    df["is_year_end"]    = ((df["month"] == 12) & (df["day_of_month"] == 31)).astype(int)
    return df

# -----------------------------------------------------------
# US holiday adjacency (pre/post holiday drift)
# -----------------------------------------------------------
def add_holiday_features(df):
    us_holidays = holidays.US()
    df["is_holiday_adjacent"] = df["date"].apply(
        lambda d: int((d + pd.Timedelta(days=1) in us_holidays) or
                      (d - pd.Timedelta(days=1) in us_holidays))
    )
    return df

# -----------------------------------------------------------
# Option expiration week (3rd Friday of each month)
# -----------------------------------------------------------
def add_opex_features(df):
    df["is_opex_week"] = df["date"].apply(
        lambda d: int((d.weekday() == 4) and (15 <= d.day <= 21))
    )
    return df

# -----------------------------------------------------------
# FOMC week (fed policy drift + volatility cluster)
# -----------------------------------------------------------
def add_fomc_features(df):
    fomc_dates = fetch_fomc_dates()   # <-- now a Series, safe for .dt
    fomc_weeks = fomc_dates.dt.isocalendar().week.unique()
    df["is_fomc_week"] = df["date"].dt.isocalendar().week.isin(fomc_weeks).astype(int)
    return df

# -----------------------------------------------------------
# High-impact macroeconomic event flags (day + week)
# -----------------------------------------------------------
def add_macro_features(df):
    macros = macro_calendar()

    for name, dates in macros.items():
        # exact date match
        df[f"is_{name}_day"] = df["date"].isin(dates).astype(int)
        # week match
        df[f"is_{name}_week"] = (
            df["date"].dt.isocalendar().week.isin(
                dates.dt.isocalendar().week.unique()
            )
        ).astype(int)

    return df

# -----------------------------------------------------------
# MASTER WRAPPER — assumes date is the index
# -----------------------------------------------------------
def add_all_calendar_features(df):
    """
    Adds all high-value seasonality and event-driven features.
    Assumes df.index is the date (yfinance default).
    """

    df = df.copy()

    # Convert index → "date" column
    df = df.reset_index().rename(columns={df.index.name or "index": "date"})
    df["date"] = pd.to_datetime(df["date"])

    # Add feature blocks
    df = add_calendar_basics(df)
    df = add_holiday_features(df)
    df = add_opex_features(df)
    df = add_fomc_features(df)
    df = add_macro_features(df)

    return df


In [50]:
data = add_all_calendar_features(data)

In [51]:
data.head(5)

,date,VIXY_Open,VIXY_High,VIXY_Low,VIXY_Close,VIXY_Volume,XPH_Open,XPH_High,XPH_Low,XPH_Close,...,is_opex_week,is_fomc_week,is_cpi_day,is_cpi_week,is_nfp_day,is_nfp_week,is_ppi_day,is_ppi_week,is_gdp_day,is_gdp_week
0,2011-01-04,633120.0,650800.0,633120.0,633840.0,14,23.285000,23.285000,22.965000,23.059999,...,0,0,0,0,0,1,0,0,0,0
1,2011-01-05,640400.0,640800.0,617440.0,620400.0,9,22.945000,23.250000,22.945000,23.219999,...,0,0,0,0,0,1,0,0,0,0
2,2011-01-06,619520.0,629440.0,614960.0,623040.0,10,23.360001,23.395000,23.209999,23.350000,...,0,0,0,0,0,1,0,0,0,0
3,2011-01-07,617920.0,640080.0,609440.0,624320.0,5,23.290001,23.400000,23.115000,23.375000,...,0,0,0,0,0,1,0,0,0,0
4,2011-01-10,637120.0,646960.0,622080.0,623040.0,9,23.370001,23.504999,23.250000,23.480000,...,0,0,0,1,0,0,0,1,0,0


In [52]:
for c in data.columns:
    print(c)

date
VIXY_Open
VIXY_High
VIXY_Low
VIXY_Close
VIXY_Volume
XPH_Open
XPH_High
XPH_Low
XPH_Close
XPH_Volume
VHT_Open
VHT_High
VHT_Low
VHT_Close
VHT_Volume
XLV_Open
XLV_High
XLV_Low
XLV_Close
XLV_Volume
PPH_Open
PPH_High
PPH_Low
PPH_Close
PPH_Volume
SPY_Open
SPY_High
SPY_Low
SPY_Close
SPY_Volume
XBI_Open
XBI_High
XBI_Low
XBI_Close
XBI_Volume
IBB_Open
IBB_High
IBB_Low
IBB_Close
IBB_Volume
IHE_Open
IHE_High
IHE_Low
IHE_Close
IHE_Volume
IBB_Return_1d
IBB_Return_5d
IBB_Return_10d
IBB_Return_20d
IBB_SMA_10
IBB_SMA_50
IBB_EMA_20
IBB_EMA_50
IBB_MA_Cross
IBB_MACD
IBB_MACD_sig
IBB_MACD_hist
IBB_RSI_14
IBB_StochK
IBB_StochD
IBB_Vol_20
IBB_Parkinson_20
IBB_GK_20
IBB_BB_width
IBB_Volume_ROC
IBB_Volume_Z
IBB_OBV
IBB_Entropy_20
IBB_HMM
IHE_Return_1d
IHE_Return_5d
IHE_Return_10d
IHE_Return_20d
IHE_SMA_10
IHE_SMA_50
IHE_EMA_20
IHE_EMA_50
IHE_MA_Cross
IHE_MACD
IHE_MACD_sig
IHE_MACD_hist
IHE_RSI_14
IHE_StochK
IHE_StochD
IHE_Vol_20
IHE_Parkinson_20
IHE_GK_20
IHE_BB_width
IHE_Volume_ROC
IHE_Volume_Z
IHE_OBV
IH

In [55]:
import pandas as pd
import numpy as np

def add_missing_features(df):
    """
    Adds missing high-value, non-leaking features.
    Automatically deduplicates the DataFrame before adding anything.
    """

    df = df.copy()

    # ----------------------------------------------------------------------
    # 0. Deduplicate columns FIRST to avoid "multiple columns returned" bugs
    # ----------------------------------------------------------------------
    df = df.loc[:, ~df.columns.duplicated(keep="last")]

    # ----------------------------------------------------------------------
    # 1. Rolling volatility / trend / autocorrelation for PPH
    # ----------------------------------------------------------------------
    close = df["PPH_Close"]
    ret = close.pct_change()

    df["PPH_vol_20"] = ret.rolling(20).std()
    df["PPH_vol_60"] = ret.rolling(60).std()
    df["PPH_vol_120"] = ret.rolling(120).std()

    df["PPH_trend_20"]  = (close - close.shift(20))  / 20
    df["PPH_trend_60"]  = (close - close.shift(60))  / 60
    df["PPH_trend_120"] = (close - close.shift(120)) / 120

    df["PPH_autocorr_20"] = ret.rolling(20).apply(lambda x: pd.Series(x).autocorr(), raw=False)
    df["PPH_autocorr_60"] = ret.rolling(60).apply(lambda x: pd.Series(x).autocorr(), raw=False)

    # ----------------------------------------------------------------------
    # 2. Drawdowns (no leakage: uses only past rolling window)
    # ----------------------------------------------------------------------
    df["PPH_drawdown_60"] = (close / close.rolling(60).max()) - 1
    df["PPH_max_drawdown_120"] = close.rolling(120).apply(
        lambda x: (x / x.cummax() - 1).min(),
        raw=False
    )

    # ----------------------------------------------------------------------
    # 3. Lagged cross-asset features (safe because we shift forward in time)
    # ----------------------------------------------------------------------
    assets = ["SPY", "VIXY", "XLV", "XBI", "IBB"]
    lags = [1, 2, 3]

    for a in assets:

        # These lines will now ALWAYS return Series (never DataFrames)
        ret_series = df[f"{a}_Return_1d"]
        vol_series = df[f"{a}_Vol_20"]

        for lag in lags:
            df[f"{a}_Return_1d_lag{lag}"] = ret_series.shift(lag)
            df[f"{a}_Vol_20_lag{lag}"] = vol_series.shift(lag)

    # ----------------------------------------------------------------------
    # 4. Regime labels
    # ----------------------------------------------------------------------
    vol20 = df["PPH_vol_20"]
    df["PPH_vol_zscore_60"] = (vol20 - vol20.rolling(60).mean()) / vol20.rolling(60).std()

    ema20 = df["PPH_EMA_20"]
    ema50 = df["PPH_EMA_50"]
    diff  = ema20 - ema50

    threshold = diff.rolling(60).std() * 0.25

    df["PPH_trend_regime"] = np.where(
        diff > threshold, 1,
        np.where(diff < -threshold, -1, 0)
    )

    vol = df["PPH_Volume"]
    df["PPH_volume_zscore_60"] = (vol - vol.rolling(60).mean()) / vol.rolling(60).std()

    # ----------------------------------------------------------------------
    # 5. Deduplicate again in case new names collided
    # ----------------------------------------------------------------------
    df = df.loc[:, ~df.columns.duplicated(keep="first")]

    return df


In [56]:
data = add_missing_features(data)

In [59]:
list(data.columns)

['date',
 'VIXY_Open',
 'VIXY_High',
 'VIXY_Low',
 'VIXY_Close',
 'VIXY_Volume',
 'XPH_Open',
 'XPH_High',
 'XPH_Low',
 'XPH_Close',
 'XPH_Volume',
 'VHT_Open',
 'VHT_High',
 'VHT_Low',
 'VHT_Close',
 'VHT_Volume',
 'XLV_Open',
 'XLV_High',
 'XLV_Low',
 'XLV_Close',
 'XLV_Volume',
 'PPH_Open',
 'PPH_High',
 'PPH_Low',
 'PPH_Close',
 'PPH_Volume',
 'SPY_Open',
 'SPY_High',
 'SPY_Low',
 'SPY_Close',
 'SPY_Volume',
 'XBI_Open',
 'XBI_High',
 'XBI_Low',
 'XBI_Close',
 'XBI_Volume',
 'IBB_Open',
 'IBB_High',
 'IBB_Low',
 'IBB_Close',
 'IBB_Volume',
 'IHE_Open',
 'IHE_High',
 'IHE_Low',
 'IHE_Close',
 'IHE_Volume',
 'IBB_Return_1d',
 'IBB_Return_5d',
 'IBB_Return_10d',
 'IBB_Return_20d',
 'IBB_SMA_10',
 'IBB_SMA_50',
 'IBB_EMA_20',
 'IBB_EMA_50',
 'IBB_MA_Cross',
 'IBB_MACD',
 'IBB_MACD_sig',
 'IBB_MACD_hist',
 'IBB_RSI_14',
 'IBB_StochK',
 'IBB_StochD',
 'IBB_Vol_20',
 'IBB_Parkinson_20',
 'IBB_GK_20',
 'IBB_BB_width',
 'IBB_Volume_ROC',
 'IBB_Volume_Z',
 'IBB_OBV',
 'IBB_Entropy_20',
 'IBB

In [58]:
data.shape

(3749, 353)

'\n✔ SHIFT(1) correctly maps f → f(t-1)\n✔ Calendar features create f(t+k) for k up to horizon\n✔ Target remains untouched\n✔ Already-lagged features remain untouched\n✔ Open features remain unshifted\n✔ Original columns that should be replaced are dropped\n✔ Defensive: skip missing columns safely\n✔ Uses clean functions for clarity\n✔ Fully commented\n'

In [61]:
horizon = 30
df_transformed = apply_all_feature_transformations(data, "feature_transformations.md", horizon=horizon)

FileNotFoundError: [Errno 2] No such file or directory: 'feature_transformations.md'

In [62]:
import pandas as pd
import re

# ============================================================
# 1. LOAD TRANSFORMATION TABLE FROM MARKDOWN
# ============================================================

def load_feature_rules(path="feature_transformations.md"):
    """
    Loads the markdown table you generated earlier and converts it into
    a clean dict: rules[feature] = {"transformation":..., "new_name":...}
    """
    # Read markdown table (pipes)
    df = pd.read_table(
        path,
        sep="|",
        engine="python",
        skiprows=2,          # Skip the markdown header separator lines
        skipinitialspace=True
    )

    # Clean up junk markdown columns
    df = df.drop(columns=[""], errors="ignore")
    df.columns = [c.strip() for c in df.columns]
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    rules = {}
    for _, row in df.iterrows():
        feature = row["feature"]
        transformation = row["transformation"]
        new_name = row["new_name"]

        rules[feature] = {
            "transformation": transformation,
            "new_name": new_name
        }

    return rules


# ============================================================
# 2. APPLY TRANSFORMATIONS USING THE RULE DICT
# ============================================================

def apply_rules_to_dataframe(df, rules, horizon=30):
    """
    Applies the transformation rules exactly as specified in the markdown table.
    """

    df = df.copy()
    to_drop = []

    for col in df.columns:

        if col not in rules:
            # If the column was not in the markdown table, we leave it unchanged
            continue

        rule = rules[col]
        trans = rule["transformation"]
        new_name = rule["new_name"]

        # ----------------------------------------------------
        # TARGET — leave untouched
        # ----------------------------------------------------
        if trans == "target":
            continue

        # ----------------------------------------------------
        # NO SHIFT
        # ----------------------------------------------------
        if trans == "no shift":
            if new_name != col:
                df[new_name] = df[col]
                to_drop.append(col)
            continue

        # ----------------------------------------------------
        # SHIFT(1)
        # ----------------------------------------------------
        if trans == "shift(1)" or trans == "shift(1)":
            df[new_name] = df[col].shift(1)
            to_drop.append(col)
            continue

        # ----------------------------------------------------
        # CALENDAR FUTURE HORIZON RULE
        # ----------------------------------------------------
        if "future horizon rule" in trans:
            # base name (col(t))
            df[new_name] = df[col]

            # generate deterministic future versions
            for k in range(1, horizon + 1):
                df[f"{col}(t+{k})"] = df[col].shift(-k)

            to_drop.append(col)
            continue

    # Drop original columns that were replaced
    df = df.drop(columns=[c for c in to_drop if c in df.columns])

    return df


# ============================================================
# 3. FULL PIPELINE
# ============================================================

def apply_all_feature_transformations(df, feature_table_path="feature_transformations.md", horizon=30):
    """
    Full pipeline:
    - loads rules from markdown file (ground truth)
    - applies them to df
    """
    rules = load_feature_rules(feature_table_path)
    updated = apply_rules_to_dataframe(df, rules, horizon=horizon)
    return updated


In [63]:
horizon = 30
data = apply_all_feature_transformations(data, feature_table_path="feature_transformations.md", horizon=horizon)

/var/folders/qc/49jwnztd1_n2k4yss03b1lkh0000gn/T/ipykernel_98756/1670282100.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


KeyError: 'feature'